# Chapter 20: Attributes and Reflection

Attributes and reflection are powerful features that enable metadata-driven programming in C#. **Attributes** allow you to add declarative information to your code – classes, methods, properties, and more. This information can then be queried at runtime using **reflection**, enabling frameworks and libraries to make decisions based on those annotations.

Think of attributes as sticky notes you attach to code elements. They don't change the behavior of the code by themselves, but other code (like serializers, test runners, or dependency injection containers) can read them and act accordingly.

In this chapter, you'll learn:

- What attributes are and how to use built‑in attributes like `[Obsolete]`, `[Serializable]`, and `[Conditional]`.
- How to create **custom attributes** by deriving from `Attribute`.
- How to restrict attribute usage with `AttributeUsage`.
- Accessing attributes at runtime via reflection.
- **Reflection** fundamentals: inspecting types, members, invoking methods dynamically.
- The performance costs of reflection and when to use it.
- A practical example: building a simple validation framework using custom attributes and reflection.
- Alternatives to reflection for high‑performance scenarios (source generators).

By the end, you'll be able to add rich metadata to your code and build tools that leverage that metadata for tasks like validation, serialization, and dependency injection.

---

## 20.1 What Are Attributes?

An **attribute** is a class that derives from `System.Attribute`. You attach it to a code element (class, method, property, etc.) by placing it in square brackets before the element.

```csharp
[Obsolete("Use NewMethod instead")]
public void OldMethod() { }
```

Attributes can have parameters. The compiler stores the attribute information in the metadata of the assembly, where it can be read later by reflection.

### Common Built‑in Attributes

| Attribute | Purpose |
|-----------|---------|
| `[Obsolete]` | Marks code as deprecated. The compiler can issue warnings or errors. |
| `[Serializable]` | Indicates that a class can be serialized (used by binary formatters, etc.). |
| `[Conditional]` | Causes method calls to be included only when a specific preprocessor symbol is defined. |
| `[CallerMemberName]`, `[CallerFilePath]`, `[CallerLineNumber]` | Used for logging and INotifyPropertyChanged to get caller information automatically. |
| `[TestMethod]` (in MSTest) | Marks a method as a unit test. |
| `[Route]` (in ASP.NET Core) | Defines URL routes for controller actions. |

---

## 20.2 Creating Custom Attributes

To create a custom attribute, define a class that inherits from `System.Attribute`.

```csharp
public class AuthorAttribute : Attribute
{
    public string Name { get; }
    public string Email { get; set; }

    public AuthorAttribute(string name)
    {
        Name = name;
    }
}
```

You can then apply it:

```csharp
[Author("Alice")]
public class SampleClass
{
    [Author("Bob", Email = "bob@example.com")]
    public void SomeMethod() { }
}
```

### Naming Convention

By convention, attribute class names end with "Attribute". When applying, you can omit the suffix: `[Author]` instead of `[AuthorAttribute]`. The compiler finds both.

### Attribute Parameters

Attributes can have two kinds of parameters:

- **Positional parameters** – defined by public constructors. They are required and appear in order.
- **Named parameters** – defined by public fields or properties. They are optional and specified with `=`.

In the example, `name` is positional, `Email` is named.

---

## 20.3 Restricting Attribute Usage with `AttributeUsage`

By default, you can apply a custom attribute to any code element. The `[AttributeUsage]` attribute lets you restrict where your attribute can be used.

```csharp
[AttributeUsage(AttributeTargets.Class | AttributeTargets.Method, AllowMultiple = false, Inherited = true)]
public class AuthorAttribute : Attribute
{
    // ...
}
```

- `AttributeTargets` – an enum with flags like `Class`, `Method`, `Property`, `Parameter`, etc.
- `AllowMultiple` – if `true`, you can apply the attribute more than once to the same element.
- `Inherited` – if `true`, the attribute is inherited by derived classes.

---

## 20.4 Accessing Attributes with Reflection

Attributes are stored in metadata. To read them at runtime, you use **reflection**. Reflection provides types like `Type`, `MethodInfo`, `PropertyInfo`, etc., that represent code elements.

### Getting Custom Attributes from a Type

```csharp
Type type = typeof(SampleClass);
object[] attributes = type.GetCustomAttributes(typeof(AuthorAttribute), inherit: true);
if (attributes.Length > 0)
{
    AuthorAttribute author = (AuthorAttribute)attributes[0];
    Console.WriteLine($"Author: {author.Name}");
}
```

Or using generic helper methods:

```csharp
AuthorAttribute author = type.GetCustomAttribute<AuthorAttribute>();
if (author != null)
{
    Console.WriteLine(author.Name);
}
```

### Getting Attributes from Methods, Properties, etc.

```csharp
MethodInfo method = type.GetMethod("SomeMethod");
AuthorAttribute methodAuthor = method.GetCustomAttribute<AuthorAttribute>();
```

### Iterating Over All Attributes

```csharp
foreach (Attribute attr in type.GetCustomAttributes())
{
    Console.WriteLine(attr.GetType().Name);
}
```

---

## 20.5 Reflection Fundamentals

Reflection allows you to inspect types and their members, and even invoke members dynamically. The entry point is usually a `Type` object, which you can obtain via:

- `typeof(MyClass)`
- `myObject.GetType()`
- `Type.GetType("Namespace.ClassName")`

### Inspecting Type Information

```csharp
Type type = typeof(string);
Console.WriteLine($"Name: {type.Name}");
Console.WriteLine($"FullName: {type.FullName}");
Console.WriteLine($"IsClass: {type.IsClass}");
Console.WriteLine($"IsValueType: {type.IsValueType}");
```

### Getting Members

```csharp
// All public methods
MethodInfo[] methods = type.GetMethods();
foreach (MethodInfo method in methods)
{
    Console.WriteLine(method.Name);
}

// Specific method
MethodInfo method = type.GetMethod("Substring", new[] { typeof(int), typeof(int) });

// Properties
PropertyInfo[] props = type.GetProperties();

// Fields (rare for public, but for private with binding flags)
FieldInfo[] fields = type.GetFields(BindingFlags.NonPublic | BindingFlags.Instance);
```

### Invoking Methods Dynamically

```csharp
object obj = Activator.CreateInstance(typeof(string), new[] { "Hello".ToCharArray() });
MethodInfo method = typeof(string).GetMethod("Substring", new[] { typeof(int), typeof(int) });
string result = (string)method.Invoke(obj, new object[] { 1, 3 });
Console.WriteLine(result); // "ell"
```

### Creating Instances

```csharp
// Using Activator (calls default constructor)
object obj = Activator.CreateInstance(typeof(StringBuilder));

// With parameters
object obj2 = Activator.CreateInstance(typeof(StringBuilder), new object[] { 100 });
```

### Getting and Setting Property Values

```csharp
Person person = new Person { Name = "Alice" };
PropertyInfo prop = typeof(Person).GetProperty("Name");
string name = (string)prop.GetValue(person);
prop.SetValue(person, "Bob");
```

Reflection is powerful but **slow** compared to normal code. Use it sparingly, and cache reflection results (e.g., `MethodInfo`, `PropertyInfo`) when possible.

---

## 20.6 Performance Considerations

Reflection involves metadata lookup and late‑bound calls, which are orders of magnitude slower than compile‑time bound code. For performance‑critical paths, consider:

- **Caching** – store `MethodInfo`, `PropertyInfo`, etc., in static dictionaries.
- **Delegate creation** – create delegates from method infos using `Delegate.CreateDelegate` for faster invocation.
- **Source generators** – in modern .NET, you can use source generators to generate code at compile time, avoiding runtime reflection. (See Chapter 29 for an introduction.)
- **Expression trees** – compile lambda expressions dynamically for repeated use.

For most business applications, reflection overhead is negligible unless called millions of times.

---

## 20.7 Putting It All Together: A Simple Validation Framework

Let's build a small validation library using custom attributes and reflection. We'll define attributes like `[Required]` and `[Range]`, and a `Validator` class that validates objects based on these attributes.

```csharp
using System;
using System.Collections.Generic;
using System.Reflection;

namespace ValidationDemo
{
    // Custom attributes
    [AttributeUsage(AttributeTargets.Property)]
    public class RequiredAttribute : Attribute { }

    [AttributeUsage(AttributeTargets.Property)]
    public class RangeAttribute : Attribute
    {
        public int Min { get; }
        public int Max { get; }
        public RangeAttribute(int min, int max)
        {
            Min = min;
            Max = max;
        }
    }

    // Model class
    public class Person
    {
        [Required]
        public string Name { get; set; }

        [Range(0, 120)]
        public int Age { get; set; }

        public string Email { get; set; }
    }

    // Validator
    public static class Validator
    {
        public static bool Validate(object obj, out List<string> errors)
        {
            errors = new List<string>();
            Type type = obj.GetType();
            foreach (PropertyInfo prop in type.GetProperties())
            {
                // Check Required
                if (prop.IsDefined(typeof(RequiredAttribute), false))
                {
                    object value = prop.GetValue(obj);
                    if (value == null || (value is string s && string.IsNullOrWhiteSpace(s)))
                    {
                        errors.Add($"{prop.Name} is required.");
                    }
                }

                // Check Range
                RangeAttribute range = prop.GetCustomAttribute<RangeAttribute>();
                if (range != null)
                {
                    object value = prop.GetValue(obj);
                    if (value is int intValue)
                    {
                        if (intValue < range.Min || intValue > range.Max)
                        {
                            errors.Add($"{prop.Name} must be between {range.Min} and {range.Max}.");
                        }
                    }
                }
            }
            return errors.Count == 0;
        }
    }

    class Program
    {
        static void Main()
        {
            Person person = new Person { Name = "", Age = 200 };
            if (!Validator.Validate(person, out List<string> errors))
            {
                foreach (string error in errors)
                {
                    Console.WriteLine(error);
                }
            }
            else
            {
                Console.WriteLine("Person is valid.");
            }
        }
    }
}
```

**Output:**
```
Name is required.
Age must be between 0 and 120.
```

This simple framework demonstrates how attributes and reflection can be used to build extensible, declarative validation logic. In a real project, you'd likely use a library like FluentValidation or data annotations, but the principle is the same.

---

## 20.8 Best Practices and Common Pitfalls

### 1. Use Attributes for Declarative Metadata, Not Behavior
Attributes should describe code, not execute it. Keep them simple; put the logic that reads attributes in separate classes (like the `Validator` above).

### 2. Name Attributes with the "Attribute" Suffix
Even though you can omit it when applying, the class name should end with "Attribute" for clarity.

### 3. Restrict Attribute Usage with `AttributeUsage`
Prevent misuse by limiting where your attribute can be applied.

### 4. Cache Reflection Results
If you repeatedly access the same type members, cache the `MethodInfo` or `PropertyInfo` objects. Reflection lookups are relatively expensive.

### 5. Consider Source Generators for Compile‑Time Alternatives
If you need high performance or compile‑time safety, explore source generators (C# 9+). They can generate code based on attributes at compile time, avoiding runtime reflection.

### 6. Be Careful with Security
Reflection can bypass access modifiers (using `BindingFlags.NonPublic`). In partially trusted environments, this may be restricted. In most full‑trust applications, it's fine but can break encapsulation.

### 7. Avoid Reflection in Performance‑Critical Paths
If you're writing a high‑performance library (e.g., a game engine), minimize reflection. Use code generation or interfaces instead.

### 8. Test Attribute Presence
When writing code that consumes attributes, always handle the case where the attribute is not present (null checks).

### 9. Use `IsDefined` for Existence Checks
If you only need to know if an attribute is present (without retrieving it), use `IsDefined` – it's slightly faster.

### 10. Document Custom Attributes
If your attributes are part of a public API, provide XML comments explaining their purpose and usage.

---

## 20.9 Chapter Summary

In this chapter, you've explored attributes and reflection:

- **Attributes** are classes that derive from `Attribute` and add metadata to code elements.
- Built‑in attributes like `[Obsolete]` and `[Serializable]` are widely used.
- **Custom attributes** let you define your own metadata, controlling usage with `AttributeUsage`.
- **Reflection** allows you to inspect types and members, read attributes, and invoke code dynamically.
- Reflection is powerful but has performance costs – cache results and consider source generators for high‑performance scenarios.
- A practical validation framework demonstrated how attributes and reflection work together.

Attributes and reflection are the backbone of many .NET features: serialization, test frameworks, dependency injection, and more. Mastering them opens the door to creating flexible, extensible systems.

In the next chapter, **Dependency Injection (DI)**, you'll learn how to decouple components using DI, one of the most important design patterns in modern application development. We'll cover the DI container, service lifetimes, and best practices.

**Exercises:**

1. Create a custom attribute `[DisplayName]` that stores a friendly name for a property. Write a method that, given an object, prints all property values along with their display names (if the attribute exists) or the property name.
2. Write a simple test runner: scan an assembly for classes with `[TestClass]` and methods with `[TestMethod]` attributes. Invoke all test methods and report success/failure.
3. Implement a `[LogExecution]` attribute that, when applied to a method, logs the method name and execution time using reflection. (Hint: you'll need to use a proxy or intercept the call – this is more advanced; you might use `DispatchProxy` or a dynamic approach.)
4. Use reflection to compare two objects of the same type and produce a list of properties that differ.
5. Create a custom serializer that uses attributes to control how properties are serialized (e.g., `[Ignore]`, `[Name("custom")]`). Serialize an object to a simple key‑value format.

Now, get ready to decouple your code with dependency injection in Chapter 21!